#**PLN - Universidad Nacional de Colombia**
## Asignatura Procesamiento de Lenguaje Natural - 2024
### Aprendizaje Supervisado:
*  Clasificación texto
*  NER - Reconocimiento de Entidades Nombradas
*  Otro

###Profesora Elizabeth León Guzmán


####Para este notebook se toma un dataset etiquetado previamente y se ha utilizado una arquitectura de redes neuronales -> modelo supervisado
---
------------------------------------------------------------------------------

# Instalaciones


In [1]:
!pip install spacy
!pip install scikit-learn
!pip install pandas

In [2]:
!pip install kagglehub

In [3]:
!pip install tensorflow

# Imports


In [4]:
import pandas as pd
import numpy as np
import spacy
spacy.prefer_gpu()
from spacy.training import offsets_to_biluo_tags
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from spacy.training import offsets_to_biluo_tags, Example
import kagglehub

import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import cross_val_score, StratifiedKFold

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# 1. Crear un modelo de reconocimiento de entidades al conjunto ner_dataset Aplicar validación cruzada.
Fuente del dataset: https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus#ner_dataset.csv

## Carga del Conjunto de Datos

En esta sección, descargamos la última versión del conjunto de datos de reconocimiento de entidades (NER) desde Kaggle y cargamos un subconjunto de 10,000 filas para facilitar el procesamiento.

1. Descargar el Conjunto de Datos
Usamos la función `dataset_download` de `kagglehub` para descargar la última versión del conjunto de datos "entity-annotated-corpus" de Kaggle. El archivo descargado es "ner_dataset.csv".

2. Leer el Archivo CSV
Utilizamos pandas para leer el archivo CSV descargado. Especificamos la codificación 'latin1' y limitamos la carga a las primeras 10,000 filas para facilitar el manejo de datos.


In [5]:
# Download latest version
path = kagglehub.dataset_download("abhinavwalia95/entity-annotated-corpus") + "/ner_dataset.csv"

# Read the CSV file
df = pd.read_csv(path, encoding='latin1', nrows=10000)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.0MB/s]

Extracting files...


## Preprocesamiento de Datos

Este fragmento de código define la función `preprocess_data`, que toma como entrada un DataFrame y convierte sus filas en una lista de oraciones estructuradas.

- **Estructura de Datos**: El DataFrame tiene columnas como "Sentence #", "Word", "POS", y "Tag".
- **Iteración y Agrupamiento**: La función itera sobre cada fila del DataFrame. Si la columna "Sentence #" es nula, agrupa las palabras junto con sus etiquetas POS y NER en una lista temporal `sentence`.
- **Cambio de Oración**: Cuando encuentra una nueva oración (indicada por un valor no nulo en "Sentence #"), agrega la oración actual a la lista `sentences` y comienza una nueva oración.
- **Resultado Final**: Al final, devuelve una lista de oraciones, donde cada oración es una lista de tuplas que contienen una palabra, su etiqueta POS, y su etiqueta NER.

Este preprocesamiento es crucial para preparar los datos de entrenamiento del modelo NER.

In [6]:
def preprocess_data(df):
    sentences = []
    sentence = []
    for i, row in df.iterrows():
        if pd.isnull(row["Sentence #"]):
            sentence.append((row["Word"], row["POS"], row["Tag"]))
        else:
            if sentence:
                sentences.append(sentence)
                sentence = []
            sentence.append((row["Word"], row["POS"], row["Tag"]))
    if sentence:
        sentences.append(sentence)
    return sentences

In [7]:
# Prepara los datos
data = preprocess_data(df)

print(data)

[[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')], [('Families', 'NNS', 'O'), ('of', 'IN', 'O'), ('soldiers', 'NNS', 'O'), ('killed', 'VBN', 'O'), ('in', 'IN', 'O'), ('the', 'DT', 'O'), ('conflict', 'NN', 'O'), ('joined', 'VBD', 'O'), ('the', 'DT', 'O'), ('protesters', 'NNS', 'O'), ('who', 'WP', 'O'), ('carried', 'VBD', 'O'), ('banners', 'NNS', 'O'), ('with', 'IN', 'O'), ('such', 'JJ', 'O'), ('slogans', 'NNS', 'O'), ('as', 'IN', 'O'), ('"', '``', 'O'), ('Bush', 'NNP', 'B-per'), ('Number', 'NN', 

### División de Datos para Validación Cruzada

En este fragmento de código, utilizamos la función `KFold` de scikit-learn para realizar la validación cruzada.

- **K-Folds**: Dividimos el conjunto de datos en 10 particiones (folds), especificado por el parámetro `n_splits=10`.
- **Propósito**: Esta técnica nos permite evaluar el rendimiento del modelo de manera más robusta, ya que entrena y evalúa el modelo múltiples veces en diferentes subconjuntos de los datos. Esto ayuda a asegurar que el modelo generaliza bien y no está sobreajustado a un solo conjunto de datos de entrenamiento.

La validación cruzada es esencial para obtener estimaciones precisas del rendimiento del modelo.

In [8]:
# Divide los datos en folds
kf = KFold(n_splits=10)

### Preparación de Datos para SpaCy y Verificación de Alineación

Este fragmento de código define la función `prepare_spacy_data`, que convierte las oraciones preprocesadas en un formato que SpaCy puede utilizar para el entrenamiento del modelo NER.

- **Transformación de Datos**: La función toma una lista de oraciones (`data`) y un objeto SpaCy (`nlp`). Para cada oración, construye una cadena de texto unida por espacios y una lista de entidades con sus posiciones inicial y final en el texto.
- **Alineación de Entidades**: Usa `offsets_to_biluo_tags` para verificar que las entidades estén correctamente alineadas con el texto. Si hay entidades desalineadas, imprime una advertencia.
- **Resultado**: Devuelve una lista de tuplas, donde cada tupla contiene un texto y un diccionario con las entidades alineadas. Este formato es necesario para entrenar el modelo NER de SpaCy.

Este preprocesamiento asegura que los datos estén en el formato correcto y que las entidades estén alineadas correctamente antes del entrenamiento del modelo.

In [9]:
# Función para preparar datos para SpaCy y verificar alineación
def prepare_spacy_data(data, nlp):
    spacy_data = []
    for sent in data:
        words = [word for word, _, _ in sent if isinstance(word, str)]
        text = " ".join(words)
        entities = []
        start = 0
        for word, _, label in sent:
            if isinstance(word, str):
                start = text.find(word, start)
                end = start + len(word)
                entities.append((start, end, label))
                start = end

        # Verifica la alineación de las entidades
        doc = nlp.make_doc(text)
        biluo_tags = offsets_to_biluo_tags(doc, entities)
        if '-' in biluo_tags:
            print(f"Warning: Misaligned entities in text '{text}'")

        spacy_data.append((text, {"entities": entities}))
    return spacy_data

### Entrenamiento y Evaluación del Modelo

En este fragmento de código, definimos dos funciones clave: `train_spacy_model` y `evaluate_model`.

- **Entrenamiento del Modelo**:
  - La función `train_spacy_model` crea un modelo NER en blanco con SpaCy y añade etiquetas para las entidades reconocidas.
  - Utiliza un optimizador para entrenar el modelo durante 10 iteraciones, actualizando el modelo con los ejemplos de entrenamiento y minimizando las pérdidas en cada iteración.
  - Devuelve el modelo entrenado.

- **Evaluación del Modelo**:
  - La función `evaluate_model` evalúa el rendimiento del modelo entrenado en un conjunto de datos de prueba.
  - Compara las etiquetas verdaderas con las etiquetas predichas y calcula las métricas de evaluación (precisión, recall y puntuación F1) usando `classification_report` de scikit-learn.
  - Asegura que las listas de etiquetas verdaderas y predichas tengan la misma longitud antes de calcular las métricas.
  - Devuelve un informe detallado con las métricas de evaluación para el modelo.

Estas funciones son esenciales para entrenar el modelo NER y evaluar su rendimiento de manera sistemática.

In [10]:
# Función para entrenar el modelo
def train_spacy_model(train_data):
    nlp = spacy.blank("en")  # Crea un modelo en blanco de SpaCy
    ner = nlp.add_pipe("ner")

    for _, annotations in train_data:
        for ent in annotations["entities"]:
            ner.add_label(ent[2])

    optimizer = nlp.begin_training()

    for itn in range(10):
        losses = {}
        for text, annotations in train_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)
        print(f"Iteration {itn}, Losses: {losses}")

    return nlp

# Función para evaluar el modelo
def evaluate_model(nlp, data):
    true_labels = []
    pred_labels = []
    for text, annotations in data:
        doc = nlp(text)
        true_labels.extend([ent[2] for ent in annotations['entities']])
        pred_labels.extend([ent.label_ for ent in doc.ents])

    # Asegúrate de que las listas sean de la misma longitud
    min_length = min(len(true_labels), len(pred_labels))
    true_labels = true_labels[:min_length]
    pred_labels = pred_labels[:min_length]

    report = classification_report(true_labels, pred_labels, output_dict=True, zero_division=0)
    return report

In [11]:
# Prepara los datos de entrenamiento y prueba
nlp = spacy.blank("en")  # Crea un modelo en blanco de SpaCy

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


### Aplicación de Validación Cruzada

En esta sección, aplicamos la validación cruzada utilizando K-Folds para evaluar el rendimiento del modelo NER en diferentes particiones del conjunto de datos.

- **División de Datos**: Dividimos los datos en índices de entrenamiento y prueba para cada fold.
- **Preparación de Datos**: Convertimos los conjuntos de entrenamiento y prueba en el formato adecuado para SpaCy utilizando la función `prepare_spacy_data`.
- **Entrenamiento y Evaluación**: Entrenamos el modelo NER en los datos de entrenamiento y evaluamos su rendimiento en los datos de prueba para cada fold.
- **Registro del Rendimiento**: Guardamos las métricas de rendimiento (precisión, recall, puntuación F1) para cada fold en la lista `fold_performance`.

Finalmente, imprimimos la precisión de cada fold para analizar cómo se desempeña el modelo en cada partición del conjunto de datos. Esto nos ayuda a obtener una estimación más robusta del rendimiento general del modelo.

In [12]:
# Aplica validación cruzada
fold_performance = []
for train_index, test_index in kf.split(data):
    train_data = [data[i] for i in train_index]
    test_data = [data[i] for i in test_index]

    # Prepara datos para SpaCy
    train_spacy_data = prepare_spacy_data(train_data, nlp)
    test_spacy_data = prepare_spacy_data(test_data, nlp)

    model = train_spacy_model(train_spacy_data)
    performance = evaluate_model(model, test_spacy_data)
    fold_performance.append(performance)

# Imprime la evaluación de cada fold
for i, performance in enumerate(fold_performance):
    print(f"Fold {i+1}:")
    print(f"Precisión: {performance['accuracy']}")
    print()

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" And I think they 'll want a one-stop shop in ter..." with entities "[(0, 1, 'O'), (2, 5, 'O'), (6, 7, 'O'), (8, 13, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Iteration 0, Losses: {'ner': 1990.8732577056362}
Iteration 1, Losses: {'ner': 1185.416065991799}
Iteration 2, Losses: {'ner': 908.8777427242054}
Iteration 3, Losses: {'ner': 797.5332857512328}
Iteration 4, Losses: {'ner': 638.2111109411561}
Iteration 5, Losses: {'ner': 544.1354359073581}
Iteration 6, Losses: {'ner': 511.91545898284784}
Iteration 7, Losses: {'ner': 470.64827029022695}
Iteration 8, Losses: {'ner': 398.9129934580913}
Iteration 9, Losses: {'ner': 408.2096552940119}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" And I think they 'll want a one-stop shop in ter..." with entities "[(0, 1, 'O'), (2, 5, 'O'), (6, 7, 'O'), (8, 13, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Iteration 0, Losses: {'ner': 2031.0089775146425}
Iteration 1, Losses: {'ner': 1206.7028578488175}
Iteration 2, Losses: {'ner': 935.6716821155856}
Iteration 3, Losses: {'ner': 738.4412362739804}
Iteration 4, Losses: {'ner': 704.5008130282296}
Iteration 5, Losses: {'ner': 609.135280219787}
Iteration 6, Losses: {'ner': 535.2197643603921}
Iteration 7, Losses: {'ner': 442.635429481121}
Iteration 8, Losses: {'ner': 414.4429259757349}
Iteration 9, Losses: {'ner': 377.77705897847363}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" And I think they 'll want a one-stop shop in ter..." with entities "[(0, 1, 'O'), (2, 5, 'O'), (6, 7, 'O'), (8, 13, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


...

Iteration 0, Losses: {'ner': 2050.011265004608}
Iteration 1, Losses: {'ner': 1146.8047025154513}
Iteration 2, Losses: {'ner': 898.2224362344906}
Iteration 3, Losses: {'ner': 780.111964728091}
Iteration 4, Losses: {'ner': 631.1701087437336}
Iteration 5, Losses: {'ner': 540.3440257023063}
Iteration 6, Losses: {'ner': 467.10812183501076}
Iteration 7, Losses: {'ner': 397.4140647835095}
Iteration 8, Losses: {'ner': 381.709173339635}
Iteration 9, Losses: {'ner': 384.20015029080184}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" And I think they 'll want a one-stop shop in ter..." with entities "[(0, 1, 'O'), (2, 5, 'O'), (6, 7, 'O'), (8, 13, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Iteration 0, Losses: {'ner': 2017.6204093671142}
Iteration 1, Losses: {'ner': 1172.6176295137739}
Iteration 2, Losses: {'ner': 923.1312547281769}
Iteration 3, Losses: {'ner': 741.7498001284825}
Iteration 4, Losses: {'ner': 630.7139279987106}
Iteration 5, Losses: {'ner': 512.1827026177996}
Iteration 6, Losses: {'ner': 471.6173578448454}
Iteration 7, Losses: {'ner': 419.9269333564279}
Iteration 8, Losses: {'ner': 430.05453151698106}
Iteration 9, Losses: {'ner': 407.64102635798105}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" And I think they 'll want a one-stop shop in ter..." with entities "[(0, 1, 'O'), (2, 5, 'O'), (6, 7, 'O'), (8, 13, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Iteration 0, Losses: {'ner': 2008.8323092994162}
Iteration 1, Losses: {'ner': 1223.8160065989482}
Iteration 2, Losses: {'ner': 964.5556120566176}
Iteration 3, Losses: {'ner': 759.3570116528155}
Iteration 4, Losses: {'ner': 649.2341501810085}
Iteration 5, Losses: {'ner': 591.8582972179909}
Iteration 6, Losses: {'ner': 484.09169774344724}
Iteration 7, Losses: {'ner': 481.15984586388333}
Iteration 8, Losses: {'ner': 371.78757831739387}
Iteration 9, Losses: {'ner': 411.20484168450173}
Fold 1:
Precisión: 0.8401826484018264

Fold 2:
Precisión: 0.8664047151277013

Fold 3:
Precisión: 0.9515463917525773

Fold 4:
Precisión: 0.7329842931937173

Fold 5:
Precisión: 0.7616777883698761

Fold 6:
Precisión: 0.9182389937106918

Fold 7:
Precisión: 0.7410358565737052

Fold 8:
Precisión: 0.930379746835443

Fold 9:
Precisión: 0.8168083097261567

Fold 10:
Precisión: 0.7652270210409745



In [13]:
# Reentrena el modelo final utilizando todos los datos
final_spacy_data = prepare_spacy_data(data, nlp)
final_model = train_spacy_model(final_spacy_data)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" And I think they 'll want a one-stop shop in ter..." with entities "[(0, 1, 'O'), (2, 5, 'O'), (6, 7, 'O'), (8, 13, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Iteration 0, Losses: {'ner': 2129.7097525005934}
Iteration 1, Losses: {'ner': 1258.6114568308067}
Iteration 2, Losses: {'ner': 1031.2050333519446}
Iteration 3, Losses: {'ner': 799.0127316864808}
Iteration 4, Losses: {'ner': 725.7327249090637}
Iteration 5, Losses: {'ner': 582.2183289566585}
Iteration 6, Losses: {'ner': 563.2266189193723}
Iteration 7, Losses: {'ner': 551.5062868491196}
Iteration 8, Losses: {'ner': 448.7544714499602}
Iteration 9, Losses: {'ner': 386.7480786450469}


### Demostración del Uso del Modelo Final

En este fragmento de código, mostramos cómo utilizar el modelo final entrenado para hacer predicciones sobre un nuevo texto.


In [14]:
# Demostración del uso del modelo final
def demonstrate_model(nlp, text):
    doc = nlp(text)
    for ent in doc.ents:
        print(f"Entidad: {ent.text}, Etiqueta: {ent.label_}")

# Texto de prueba para demostración
# Test text for demonstration
test_text = """
Albert Einstein visited the Smithsonian Institution in Washington D.C. in 1935 to view the Apollo Lunar Module.
The United Nations held a conference in New York City to discuss climate change on Earth Day.
The Great Barrier Reef, one of the seven natural wonders, is located off the coast of Queensland, Australia.
In 2019, the Louvre Museum in Paris exhibited the Mona Lisa, an artifact of immense historical value.
During the Super Bowl in Miami, fans witnessed one of the most thrilling sports events ever.
The World Health Organization announced new health guidelines last month to combat the spread of diseases.
Mount Everest, the highest peak in the world, continues to attract climbers from all over the globe.
"""
demonstrate_model(final_model, test_text)

Entidad: Albert, Etiqueta: I-per
Entidad: Einstein, Etiqueta: I-per
Entidad: visited, Etiqueta: O
Entidad: the, Etiqueta: O
Entidad: Smithsonian, Etiqueta: B-gpe
Entidad: Institution, Etiqueta: I-org
Entidad: in, Etiqueta: O
Entidad: Washington, Etiqueta: B-geo
Entidad: D.C., Etiqueta: I-geo
Entidad: in, Etiqueta: O
Entidad: 1935, Etiqueta: B-tim
Entidad: to, Etiqueta: O
Entidad: view, Etiqueta: O
Entidad: the, Etiqueta: O
Entidad: Apollo, Etiqueta: B-per
Entidad: Lunar, Etiqueta: I-per
Entidad: Module, Etiqueta: I-per
Entidad: ., Etiqueta: O
Entidad: The, Etiqueta: O
Entidad: United, Etiqueta: B-org
Entidad: Nations, Etiqueta: I-org
Entidad: held, Etiqueta: O
Entidad: a, Etiqueta: O
Entidad: conference, Etiqueta: O
Entidad: in, Etiqueta: O
Entidad: New, Etiqueta: B-geo
Entidad: York, Etiqueta: I-geo
Entidad: City, Etiqueta: I-geo
Entidad: to, Etiqueta: O
Entidad: discuss, Etiqueta: O
Entidad: climate, Etiqueta: O
Entidad: change, Etiqueta: O
Entidad: on, Etiqueta: O
Entidad: Earth, Et

Para concluir la primera parte del taller cabe resaltar que el modelo es bastante mejorable. En previas iteraciones se utilizaron más datos para el entrenamiento pero esto no mejoró el modelo, únicamente hizo el proceso de entrenamiento más tardado. Para futuras iteraciones se podría considerar lo siguiente:

1. **Ajuste de Hiperparámetros**: Experimenta con diferentes hiperparámetros como la tasa de aprendizaje, el número de iteraciones, y el factor de abandono (dropout) para mejorar el rendimiento del modelo.

2. **Embeddings Preentrenados**: Utiliza embeddings preentrenados como word2vec, GloVe o BERT que pueden proporcionar representaciones de palabras más ricas y precisas.

3. **Balance de Clases**: Asegúrate de que las clases estén equilibradas para evitar sesgos en el modelo. Puedes hacerlo mediante sobremuestreo de clases minoritarias o submuestreo de clases mayoritarias.

4. **Limpieza de Datos**: Revisa y limpia tu conjunto de datos para eliminar ruido y correcciones erróneas. Datos más limpios pueden llevar a un modelo más preciso.

5. **Ensamblado de Modelos**: Combina múltiples modelos para formar un modelo ensemble que puede mejorar la precisión al aprovechar las fortalezas de diferentes modelos individuales.

6. **Entrenamiento Incremental**: Realiza un entrenamiento incremental donde el modelo se entrena en lotes pequeños y sucesivos de datos, permitiendo ajustes finos y mejoras continuas.

#2. Construir varios modelos de análisis de sentimientos. Se deberán utilizar diferentes representaciones de los documentos y diferentes modelos de aprendizaje maquinal.
Usar el dataset para análisis de sentimientos de Kaggle, que busca identificar contenido odioso en tweets.

In [15]:
# Descarga de la base de datos
path = kagglehub.dataset_download("philipsanm/sentiment-analysis-in-spanish-tweets") + "/sentiment_analysis_dataset.csv"

df = pd.read_csv(path)

100%|██████████| 181k/181k [00:00<00:00, 384kB/s]

Extracting files...


In [16]:
# Preprocesamiento del texto: limpieza, tokenización, etc.
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

def preprocess_text(text):
    text = re.sub(r'\W', ' ', str(text))
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub(r'^b\s+', '', text)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stop_words]
    text = ' '.join(text)
    return text

df['text'] = df['text'].apply(preprocess_text)
print(df['text'])

[nltk_data] Downloading package stopwords to /root/nltk_data...


0                       termine bien abrumado después hoy
1                                         siento abrumado
2       siento abrumado cantidad cosas quiero dibujar ...
3       salvador única persona abrumado versiones nadi...
4                         denme helado ando full abrumado
                              ...                        
2585    podemos vivir miedo manejen borrachos dejen us...
2586                           vida constante miedo éxito
2587    esquizofrenia mente dividida miedo realidades ...
2588       miedo da ver cómo desapareces mundo temo mundo
2589       saltando apoco ala pile agua xd miedo relajado
Name: text, Length: 2590, dtype: object


[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
# Representación de los documentos usando TF-IDF
tfidf_vect = TfidfVectorizer(max_features=5000)
X = tfidf_vect.fit_transform(df['text']).toarray()
y = df['sentiment']  # O df['emotion']
print(X)
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0       scared
1       scared
2       scared
3       scared
4       scared
         ...  
2585    joyful
2586    joyful
2587    joyful
2588    joyful
2589    joyful
Name: sentiment, Length: 2590, dtype: object


### Modelo Naive Bayes

Naive Bayes es un algoritmo de clasificación basado en el teorema de Bayes. Se llama "naive" (ingenuo) porque asume que las características de entrada son independientes entre sí, lo que rara vez es el caso en la práctica. Sin embargo, esta simplificación permite que el modelo sea muy eficiente y rápido.

#### Ventajas:
1. **Simplicidad y Rapidez**: Es fácil de implementar y puede entrenarse rápidamente incluso con grandes conjuntos de datos.
2. **Poca Memoria**: Requiere menos memoria para almacenar el modelo en comparación con otros algoritmos.
3. **Rendimiento Decente**: A pesar de sus suposiciones ingenuas, a menudo proporciona un rendimiento competitivo en problemas de clasificación, especialmente en textos.
4. **Probabilístico**: Genera probabilidades de pertenencia a clase, lo que puede ser útil para tareas donde es importante conocer la confianza del modelo.

Este modelo es ideal para tareas de clasificación de texto, como el análisis de sentimientos, donde la relación entre las palabras (características) y las etiquetas de clase puede modelarse efectivamente con esta técnica probabilística.

In [18]:
# Modelo Naive Bayes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      joyful       0.69      0.18      0.29        61
         mad       0.64      0.72      0.68       106
    peaceful       0.43      0.87      0.57       135
    powerful       0.68      0.39      0.50        92
         sad       0.74      0.43      0.54        68
      scared       0.67      0.21      0.32        56

    accuracy                           0.54       518
   macro avg       0.64      0.47      0.48       518
weighted avg       0.61      0.54      0.52       518



### Modelo SVM (Support Vector Machine)

El modelo SVM, o Máquinas de Vectores de Soporte, es un poderoso algoritmo de clasificación que busca encontrar un hiperplano en un espacio de múltiples dimensiones que separa las clases de la mejor manera posible. Este modelo es conocido por su capacidad para manejar tanto problemas lineales como no lineales mediante el uso de kernels.

#### Ventajas:
1. **Eficiencia en Espacios de Alta Dimensión**: SVM es eficaz en espacios de alta dimensión y cuando el número de dimensiones es mayor que el número de muestras.
2. **Manejo de Casos Lineales y No Lineales**: Puede ser usado tanto para clasificación lineal como para clasificación no lineal mediante el uso de la función kernel adecuada.
3. **Robustez Frente a Overfitting**: Especialmente útil en casos donde hay una clara separación de clases y ayuda a minimizar el overfitting en altas dimensiones.
4. **Flexibilidad de Kernels**: Permite el uso de múltiples kernels (lineal, polinómico, radial, etc.) que pueden adaptarse a diferentes tipos de problemas.

Las SVM son ampliamente utilizadas en aplicaciones de reconocimiento de patrones y clasificación de texto, donde su capacidad para encontrar la separación óptima entre clases puede llevar a modelos de gran precisión y robustez.


In [19]:
# Modelo SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

      joyful       0.72      0.56      0.63        61
         mad       0.82      0.76      0.79       106
    peaceful       0.65      0.82      0.73       135
    powerful       0.69      0.72      0.71        92
         sad       0.77      0.69      0.73        68
      scared       0.83      0.68      0.75        56

    accuracy                           0.73       518
   macro avg       0.75      0.71      0.72       518
weighted avg       0.74      0.73      0.73       518



### Modelo Random Forest

Random Forest es un algoritmo de ensamble que crea un "bosque" de árboles de decisión independientes, y su decisión final se basa en la combinación de las predicciones de estos múltiples árboles. Cada árbol en el bosque se entrena con una muestra diferente del conjunto de datos y el algoritmo promedio de las predicciones de todos los árboles para tomar la decisión final.

#### Ventajas:
1. **Reducción del Overfitting**: Al promediar las predicciones de múltiples árboles, Random Forest reduce el riesgo de sobreajuste que puede ocurrir con un solo árbol de decisión.
2. **Robustez y Estabilidad**: Es menos sensible a los valores atípicos y a las variaciones en el conjunto de datos, lo que lo hace un modelo muy robusto.
3. **Manejo de Datos No Lineales**: Puede capturar relaciones complejas y no lineales entre las características y las etiquetas de clase.
4. **Flexibilidad**: Puede ser utilizado tanto para problemas de clasificación como de regresión.
5. **Importancia de las Variables**: Proporciona una medida de la importancia de cada característica en la predicción, lo cual es útil para entender el modelo.

Random Forest es una excelente opción para tareas de clasificación como el análisis de sentimientos debido a su capacidad para manejar conjuntos de datos complejos y su alto rendimiento de predicción.

In [20]:
# Modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

      joyful       0.80      0.70      0.75        61
         mad       0.91      0.75      0.82       106
    peaceful       0.74      0.90      0.81       135
    powerful       0.84      0.76      0.80        92
         sad       0.77      0.79      0.78        68
      scared       0.86      0.91      0.89        56

    accuracy                           0.81       518
   macro avg       0.82      0.80      0.81       518
weighted avg       0.82      0.81      0.81       518



### Modelo Logistic Regression

La regresión logística es un modelo estadístico utilizado para la clasificación binaria que predice la probabilidad de pertenencia de una instancia a una de dos clases posibles. Aunque su nombre sugiere que se trata de un modelo de regresión, en realidad se utiliza principalmente para tareas de clasificación.

#### Ventajas:
1. **Interpretabilidad**: Los coeficientes del modelo proporcionan una intuición clara sobre la relación entre las características y la probabilidad de la clase objetivo.
2. **Simplicidad**: Es sencillo de implementar y rápido de entrenar, especialmente en conjuntos de datos grandes.
3. **Eficacia en Espacios Lineales**: Funciona muy bien cuando las clases son linealmente separables.
4. **Robustez a la Multicolinealidad**: Aunque la multicolinealidad puede ser un problema, la regresión logística es relativamente robusta y puede manejar colinealidades moderadas entre las características.
5. **Extensibilidad**: Puede extenderse fácilmente para problemas multiclase utilizando técnicas como One-vs-Rest (OvR) o Softmax Regression.

La regresión logística es una herramienta poderosa y ampliamente utilizada en problemas de clasificación como el análisis de sentimientos, donde su capacidad para predecir probabilidades de clase puede proporcionar insights adicionales sobre la confianza de las predicciones del modelo.


In [21]:
# Modelo Logistic Regression
logreg_model = LogisticRegression(max_iter=1000, random_state=42)
logreg_model.fit(X_train, y_train)

y_pred_logreg = logreg_model.predict(X_test)
print(classification_report(y_test, y_pred_logreg))

              precision    recall  f1-score   support

      joyful       0.70      0.38      0.49        61
         mad       0.76      0.70      0.73       106
    peaceful       0.51      0.84      0.64       135
    powerful       0.72      0.61      0.66        92
         sad       0.71      0.59      0.65        68
      scared       0.77      0.41      0.53        56

    accuracy                           0.64       518
   macro avg       0.69      0.59      0.62       518
weighted avg       0.67      0.64      0.63       518



### Modelo Gradient Boosting

Gradient Boosting es un algoritmo de ensamble que construye un modelo fuerte a partir de múltiples modelos débiles, típicamente árboles de decisión. Funciona de manera iterativa, ajustando los errores residuales de los modelos anteriores en cada nueva iteración para mejorar la precisión general del modelo.

#### Ventajas:
1. **Precisión Elevada**: Gradient Boosting es conocido por su alta precisión y capacidad para manejar datos complejos y no lineales.
2. **Flexibilidad**: Puede usarse tanto para problemas de clasificación como de regresión, y permite ajustar una amplia gama de hiperparámetros para optimizar el modelo.
3. **Manejo de Sesgo y Varianza**: Reduce tanto el sesgo como la varianza del modelo al enfocarse en los errores residuales de iteraciones anteriores, mejorando gradualmente el rendimiento del modelo.
4. **Robustez a Overfitting**: Mediante técnicas como el subsampling y la regularización, Gradient Boosting puede prevenir el sobreajuste, especialmente en conjuntos de datos con mucho ruido.

Gradient Boosting es una opción poderosa y versátil para el análisis de sentimientos, ofreciendo un equilibrio excelente entre precisión, robustez y flexibilidad en el ajuste del modelo. Esto lo convierte en una herramienta muy valiosa para capturar patrones complejos y mejorar continuamente las predicciones.

In [22]:
# Modelo Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)
print(classification_report(y_test, y_pred_gb))

              precision    recall  f1-score   support

      joyful       0.89      0.69      0.78        61
         mad       0.94      0.76      0.84       106
    peaceful       0.70      0.92      0.79       135
    powerful       0.85      0.72      0.78        92
         sad       0.82      0.79      0.81        68
      scared       0.86      0.96      0.91        56

    accuracy                           0.81       518
   macro avg       0.84      0.81      0.82       518
weighted avg       0.83      0.81      0.81       518



### Modelo LSTM (Long Short-Term Memory)

Las LSTM, o Memorias de Corto y Largo Plazo, son una variante de las redes neuronales recurrentes (RNN) diseñadas específicamente para capturar dependencias a largo plazo en secuencias de datos. Son especialmente útiles en tareas de procesamiento de lenguaje natural (NLP) donde el contexto a largo plazo puede ser crucial para hacer predicciones precisas.

#### Ventajas:
1. **Captura de Dependencias a Largo Plazo**: Las LSTM pueden recordar información durante largos periodos de tiempo, lo que las hace ideales para secuencias de texto donde el contexto distante es importante.
2. **Manejo de Desvanecimiento del Gradiente**: A diferencia de las RNN tradicionales, las LSTM están diseñadas para mitigar el problema del desvanecimiento del gradiente, permitiendo que aprendan mejor en secuencias largas.
3. **Flexibilidad y Adaptabilidad**: Pueden ajustarse y adaptarse bien a diferentes tipos de datos secuenciales, no solo texto, sino también series temporales y otros datos secuenciales.
4. **Robustez a Datos Ruido**: Gracias a sus celdas de memoria y puertas de control, las LSTM pueden filtrar información irrelevante y centrarse en los datos importantes.

Las LSTM son particularmente eficaces en tareas de análisis de sentimientos debido a su capacidad para capturar el contexto a largo plazo y entender la relación entre palabras y frases en una secuencia de texto. Esto las convierte en una herramienta poderosa para mejorar la precisión de las predicciones en problemas de clasificación de textos.

In [23]:
# Tokenización y preparación de secuencias
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['text'])
X = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(X, maxlen=100)
y = pd.get_dummies(df['sentiment']).values  # O df['emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo LSTM
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=5000, output_dim=128))
lstm_model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(units=y.shape[1], activation='softmax'))

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

y_pred_lstm = lstm_model.predict(X_test)
y_pred_lstm_labels = np.argmax(y_pred_lstm, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

print(classification_report(y_test_labels, y_pred_lstm_labels))

Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 18s 340ms/step - accuracy: 0.2174 - loss: 1.7710 - val_accuracy: 0.2072 - val_loss: 1.7525
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 12s 239ms/step - accuracy: 0.2633 - loss: 1.7205 - val_accuracy: 0.3398 - val_loss: 1.7079
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 8s 162ms/step - accuracy: 0.4294 - loss: 1.5828 - val_accuracy: 0.4072 - val_loss: 1.5201
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 241ms/step - accuracy: 0.7214 - loss: 1.1017 - val_accuracy: 0.5518 - val_loss: 1.3126
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 180ms/step - accuracy: 0.8778 - loss: 0.5712 - val_accuracy: 0.5735 - val_loss: 1.2443
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
              precision    recall  f1-score   support

           0       0.53      0.33      0.40        61
           1       0.53      0.75      0.62       106
           2       0.64      0.71      0.67       135
           3       0.54      0.52      0.53        92
           4       0.54      0.43      0.48        6

De los resultados anteriores, el mejor de los modelos fue el modelo de Gradiente Boosting así que vamos a realizar el etiquetado de un texto nuevo para comprobar su rendimiento.

In [24]:
df = pd.read_csv(path)
df['text'] = df['text'].apply(preprocess_text)
tfidf_vect = TfidfVectorizer(max_features=5000)
X = tfidf_vect.fit_transform(df['text']).toarray()
y = df['sentiment']  # O df['emotion']

In [25]:
# Codificar las etiquetas de sentimiento
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Ejemplo de texto de Twitter
tweet_text = "¡Qué hermoso día! Estoy muy feliz de poder disfrutar del sol y la naturaleza. 🌞🌳 #Feliz #Contento"

# Preprocesar el texto del tweet
tweet_text_processed = preprocess_text(tweet_text)

# Transformar el texto del tweet usando TF-IDF
tweet_tfidf = tfidf_vect.transform([tweet_text_processed])

# Predecir el sentimiento usando el modelo de Gradient Boosting
sentiment_prediction = gb_model.predict(tweet_tfidf)

# Interpretar la predicción asegurando consistencia con las etiquetas originales
sentiment_label = label_encoder.inverse_transform([sentiment_prediction[0]])[0]

print(f"Texto del tweet: {tweet_text}")
print(f"Predicción de sentimiento: {sentiment_label}")

Texto del tweet: ¡Qué hermoso día! Estoy muy feliz de poder disfrutar del sol y la naturaleza. 🌞🌳 #Feliz #Contento
Predicción de sentimiento: peaceful


# 3. Cargar el conjunto de datos 20 news group. Realizar un modelo de clasificación de documentos. Usar 10% del conjunto para testing. El 90% usarlo para generar y validar el modelo. Usar validación cruzada de folding=10

In [26]:
# Cargar el conjunto de datos con metadata removido
newsgroups = fetch_20newsgroups(subset='all', shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

**Definir los Datos y las Etiquetas:**
- `X, y = newsgroups.data, newsgroups.target`
  - Este fragmento de código asigna las noticias (`data`) y sus categorías (`target`) del conjunto de datos 20 Newsgroups a las variables `X` (textos de las noticias) y `y` (etiquetas de las noticias), respectivamente.

**Dividir los Datos en Conjuntos de Prueba y Entrenamiento/Validación:**
- `X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)`
  - Aquí, los datos se dividen en dos conjuntos: el 90% se usa para entrenamiento y validación (`X_train_valid` y `y_train_valid`), y el 10% restante se utiliza para prueba (`X_test` y `y_test`). La división es estratificada (`stratify=y`) para asegurar que cada conjunto mantenga la misma proporción de clases, y se utiliza una semilla aleatoria fija (`random_state=42`) para reproducibilidad.

In [27]:
# Definir los datos y las etiquetas
X, y = newsgroups.data, newsgroups.target

# Dividir los datos en conjunto de prueba (10%) y conjunto de entrenamiento/validación (90%)
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

**Representación de los Documentos Usando TF-IDF:**
- `tfidf_vect = TfidfVectorizer(max_features=5000)`
  - Esta línea de código inicializa el vectorizador TF-IDF (Term Frequency-Inverse Document Frequency) con un límite máximo de 5000 características (palabras más importantes).
  
- `X_train_valid_tfidf = tfidf_vect.fit_transform(X_train_valid)`
  - Aquí, el vectorizador se ajusta a los datos de entrenamiento/validación (`X_train_valid`) y transforma esos textos en representaciones numéricas TF-IDF, almacenándolos en `X_train_valid_tfidf`.

- `X_test_tfidf = tfidf_vect.transform(X_test)`
  - Finalmente, el mismo vectorizador se utiliza para transformar el conjunto de prueba (`X_test`) en representaciones numéricas TF-IDF, asegurando que se utilice la misma configuración de características que se ajustó en el conjunto de entrenamiento/validación.

In [28]:
# Representación de los documentos usando TF-IDF
tfidf_vect = TfidfVectorizer(max_features=5000)
X_train_valid_tfidf = tfidf_vect.fit_transform(X_train_valid)
X_test_tfidf = tfidf_vect.transform(X_test)

**Validación Cruzada con K-Folding (k=10):**
- `kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)`
  - Esta línea de código crea un objeto `StratifiedKFold` para realizar validación cruzada estratificada con 10 folds. Estratificación significa que cada fold mantendrá la proporción de clases, asegurando una mejor representación de los datos. `shuffle=True` asegura que los datos se mezclen aleatoriamente antes de la división, y `random_state=42` se utiliza para garantizar la reproducibilidad de los resultados.

**Definición del Modelo:**
- `model = MultinomialNB()`
  - Aquí, se define el modelo de Naive Bayes Multinomial, una variante del algoritmo Naive Bayes que es especialmente adecuado para datos de conteo, como texto representado mediante TF-IDF o frecuencias de palabras. Este modelo será entrenado y evaluado utilizando la validación cruzada.

In [29]:
# Validación cruzada con k-folding (k=10)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
model = MultinomialNB()

**Entrenar y Validar el Modelo:**
- `cross_val_scores = cross_val_score(model, X_train_valid_tfidf, y_train_valid, cv=kfold, scoring='accuracy')`
  - Esta línea realiza la validación cruzada utilizando `cross_val_score`, aplicando el modelo de Naive Bayes Multinomial (`model`) a los datos de entrenamiento/validación transformados por TF-IDF (`X_train_valid_tfidf`). Se emplea el método de k-folding (con `cv=kfold`), con 10 folds, para evaluar el modelo. La métrica de evaluación utilizada es la precisión (`scoring='accuracy'`).

- `print(f"Validación cruzada (10 folds): {np.mean(cross_val_scores):.4f} ± {np.std(cross_val_scores):.4f}")`
  - Aquí, se imprime la media y la desviación estándar de las puntuaciones de precisión obtenidas a partir de la validación cruzada con 10 folds. Esto proporciona una idea del rendimiento promedio del modelo y su variabilidad.

**Entrenar el Modelo Final en el Conjunto de Entrenamiento/Validación Completo:**
- `model.fit(X_train_valid_tfidf, y_train_valid)`
  - Finalmente, el modelo de Naive Bayes Multinomial se entrena utilizando el conjunto completo de entrenamiento/validación transformado por TF-IDF (`X_train_valid_tfidf`) y sus etiquetas (`y_train_valid`). Esto permite que el modelo se ajuste a todos los datos disponibles, maximizando su capacidad de generalización antes de evaluar su rendimiento en el conjunto de prueba.

In [30]:
# Entrenar y validar el modelo
cross_val_scores = cross_val_score(model, X_train_valid_tfidf, y_train_valid, cv=kfold, scoring='accuracy')
print(f"Validación cruzada (10 folds): {np.mean(cross_val_scores):.4f} ± {np.std(cross_val_scores):.4f}")

# Entrenar el modelo final en el conjunto de entrenamiento/validación completo
model.fit(X_train_valid_tfidf, y_train_valid)

Validación cruzada (10 folds): 0.6799 ± 0.0101


MultinomialNB()

**Evaluar el Modelo en el Conjunto de Prueba:**
- `y_pred = model.predict(X_test_tfidf)`
  - Esta línea de código utiliza el modelo entrenado para predecir las etiquetas de los documentos en el conjunto de prueba (`X_test_tfidf`). Las predicciones se almacenan en `y_pred`.

- `print("Evaluación del modelo en el conjunto de prueba:")`
  - Aquí se imprime un mensaje indicando que comienza la evaluación del modelo en el conjunto de prueba.

- `print(classification_report(y_test, y_pred, target_names=newsgroups.target_names))`
  - Esta línea genera e imprime un informe de clasificación que muestra las métricas de rendimiento del modelo, como precisión, recall y F1-score, para cada una de las categorías del conjunto de datos 20 Newsgroups. El informe utiliza las etiquetas verdaderas (`y_test`) y las etiquetas predichas (`y_pred`), y se organiza con nombres de las categorías (`target_names`).

In [31]:
# Evaluar el modelo en el conjunto de prueba
y_pred = model.predict(X_test_tfidf)
print("Evaluación del modelo en el conjunto de prueba:")
print(classification_report(y_test, y_pred, target_names=newsgroups.target_names))

Evaluación del modelo en el conjunto de prueba:
                          precision    recall  f1-score   support

             alt.atheism       0.64      0.36      0.46        80
           comp.graphics       0.69      0.70      0.70        97
 comp.os.ms-windows.misc       0.68      0.55      0.60        99
comp.sys.ibm.pc.hardware       0.60      0.76      0.67        98
   comp.sys.mac.hardware       0.75      0.66      0.70        96
          comp.windows.x       0.78      0.82      0.80        99
            misc.forsale       0.84      0.69      0.76        98
               rec.autos       0.66      0.71      0.68        99
         rec.motorcycles       0.80      0.59      0.68       100
      rec.sport.baseball       0.83      0.79      0.81        99
        rec.sport.hockey       0.55      0.87      0.67       100
               sci.crypt       0.78      0.78      0.78        99
         sci.electronics       0.72      0.68      0.70        98
                 sci.med   

Si bien el rendimiento del modelo no es excepcional, en otra versión del código se utilizó Gradient Boosting y por el volumen de los datos el entrenamiento fue muy lento. Este proceso es identico o muy similar a la hora de usar otros modelos como los usados en el punto anterior, así que con algo más de potencia computacional y tiempo se podría usar una arquitectura diferente para mejorar resultados.

#4. Modelos geenrados con LSTM para generar texto (extensión del notebook visto en clase de LSTM)

**Cargar Texto ASCII y Convertir a Minúsculas:**

- `filename = "wonderland.txt"`
  - Esta línea de código define el nombre del archivo que contiene el texto ASCII. En este caso, el archivo es "wonderland.txt".

- `raw_text = open(filename, 'r', encoding='utf-8').read()`
  - Aquí, el archivo especificado se abre en modo de lectura (`'r'`) con codificación UTF-8. Luego, el contenido del archivo se lee completamente y se almacena en la variable `raw_text`.

- `raw_text = raw_text.lower()`
  - Finalmente, todo el texto almacenado en `raw_text` se convierte a minúsculas para asegurar uniformidad en el procesamiento posterior. Esto es útil para evitar diferencias entre mayúsculas y minúsculas durante el análisis y generación de texto.

In [49]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

**Crear Mapeo de Caracteres Únicos a Enteros:**

- `chars = sorted(list(set(raw_text)))`
  - Esta línea de código identifica todos los caracteres únicos en el texto `raw_text`. Primero, convierte el texto en un conjunto (`set`) para eliminar duplicados, luego convierte el conjunto a una lista (`list`) y la ordena alfabéticamente (`sorted`). El resultado es una lista de caracteres únicos ordenados que se almacena en la variable `chars`.

- `char_to_int = dict((c, i) for i, c in enumerate(chars))`
  - Aquí, se crea un diccionario `char_to_int` que mapea cada carácter único (`c`) a un entero (`i`). La función `enumerate(chars)` asigna un índice a cada carácter en la lista `chars`, y el diccionario se construye mediante una comprensión de diccionario, asignando cada carácter a su correspondiente índice. Este mapeo es útil para convertir caracteres en representaciones numéricas que pueden ser procesadas por el modelo LSTM.


In [50]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

**Calcular el Número Total de Caracteres y el Tamaño del Vocabulario:**

- `n_chars = len(raw_text)`
  - Esta línea de código calcula el número total de caracteres en el texto `raw_text` y almacena ese valor en la variable `n_chars`.

- `n_vocab = len(chars)`
  - Aquí, se calcula el tamaño del vocabulario, es decir, el número de caracteres únicos en la lista `chars`, y se almacena en la variable `n_vocab`.

- `print("Total Characters: ", n_chars)`
  - Esta línea imprime el total de caracteres en el texto.

- `print("Total Vocab: ", n_vocab)`
  - Finalmente, esta línea imprime el tamaño del vocabulario, mostrando cuántos caracteres únicos hay en el texto.

Este proceso ayuda a tener una visión general de la cantidad de datos y la diversidad de caracteres en el texto, lo cual es crucial para tareas de generación de texto.

In [51]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163919
Total Vocab:  65


**Preparar el Conjunto de Datos de Pares de Entrada y Salida Codificados como Enteros:**

- `seq_length = 100`
  - Define la longitud de las secuencias de entrada (`seq_length`) como 100 caracteres.

- `dataX = []` y `dataY = []`
  - Inicializa dos listas vacías, `dataX` y `dataY`, para almacenar las secuencias de entrada y sus correspondientes salidas, respectivamente.

- Bucle `for`:
  ```python
  for i in range(0, n_chars - seq_length, 1):
      seq_in = raw_text[i:i + seq_length]
      seq_out = raw_text[i + seq_length]
      dataX.append([char_to_int[char] for char in seq_in])
      dataY.append(char_to_int[seq_out])
  ```
  - Este bucle recorre el texto `raw_text` desde el primer carácter hasta el final menos la longitud de la secuencia (`n_chars - seq_length`), con un paso de 1.
  - `seq_in = raw_text[i:i + seq_length]` extrae una secuencia de `seq_length` caracteres del texto de entrada.
  - `seq_out = raw_text[i + seq_length]` extrae el carácter que sigue inmediatamente a la secuencia de entrada.
  - `dataX.append([char_to_int[char] for char in seq_in])` convierte cada carácter de la secuencia de entrada en su representación numérica y lo añade a la lista `dataX`.
  - `dataY.append(char_to_int[seq_out])` convierte el carácter de salida en su representación numérica y lo añade a la lista `dataY`.

- `n_patterns = len(dataX)`
  - Calcula el número total de patrones (pares de entrada y salida) generados y lo almacena en `n_patterns`.

- `print("Total Patterns: ", n_patterns)`
  - Imprime el número total de patrones generados.

Este código prepara el conjunto de datos para entrenamiento del modelo LSTM, donde cada secuencia de entrada de 100 caracteres está asociada con un carácter de salida, todos codificados como enteros.

In [52]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163819


**Reformatear `X` para que sea [samples, time steps, features]:**
- `X = np.reshape(dataX, (n_patterns, seq_length, 1))`
  - Esta línea de código convierte la lista `dataX` en un arreglo NumPy con las dimensiones adecuadas para entrenar el modelo LSTM. La forma del arreglo es `(n_patterns, seq_length, 1)`, donde `n_patterns` es el número de secuencias, `seq_length` es la longitud de cada secuencia, y `1` indica que cada característica es un valor escalar.

**Normalizar `X`:**
- `X = X / float(n_vocab)`
  - Aquí, cada valor en `X` se divide por `n_vocab`, el tamaño del vocabulario, para normalizar los valores entre 0 y 1. Esta normalización ayuda a mejorar la convergencia del modelo durante el entrenamiento.

**Codificar la Variable de Salida con One-Hot Encoding:**
- `y = to_categorical(dataY)`
  - Esta línea convierte la lista `dataY`, que contiene las etiquetas de salida como enteros, en una representación one-hot encoded usando la función `to_categorical`. Esto transforma `y` en una matriz binaria donde cada fila corresponde a una etiqueta de salida con un único `1` en la posición de la categoría correcta y `0` en todas las demás posiciones. Esta representación es necesaria para la salida `softmax` del modelo LSTM.

In [53]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

**Modelo LSTM Mejorado:**

1. **Inicializar el Modelo:**
   - Se crea un modelo secuencial de Keras.

2. **Primera Capa LSTM:**
   - Se añade una capa LSTM con 256 unidades que devuelve secuencias completas (`return_sequences=True`). La entrada tiene una forma específica (longitud de la secuencia y número de características).

3. **Primera Capa de Dropout:**
   - Añade una capa de Dropout con una tasa de 0.2 para prevenir el sobreajuste.

4. **Segunda Capa LSTM:**
   - Se añade otra capa LSTM con 256 unidades para captar patrones más complejos.

5. **Segunda Capa de Dropout:**
   - Añade otra capa de Dropout con una tasa de 0.2.

6. **Capa de Salida Dense:**
   - Se agrega una capa de salida con activación `softmax` para producir una distribución de probabilidad sobre las clases posibles.

7. **Compilar el Modelo:**
   - El modelo se compila usando `categorical_crossentropy` como función de pérdida y `adam` como optimizador.

8. **Resumen del Modelo:**
   - Se muestra un resumen de la arquitectura del modelo, incluyendo el número de parámetros y las formas de las capas.

Este modelo debería ofrecer una capacidad mejorada para aprender y generalizar patrones en los datos de texto.

In [54]:
# Modelo mejorado
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

print(model.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                        │ (None, 100, 256)            │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │          16,448 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 805,952 (3.07 MB)

 Trainable params: 805,952 (3.07 MB)

 Non-trainable params: 0 (0.00 B)

None


**Definir el Checkpoint:**

- `filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras"`
  - Esta línea define el patrón de nombre para los archivos de checkpoint que se guardarán. El nombre del archivo incluirá el número de época (`epoch`) y la pérdida (`loss`) con cuatro decimales de precisión. Por ejemplo, un archivo podría llamarse "weights-improvement-01-0.1234.keras".

- `checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')`
  - Aquí se crea un callback `ModelCheckpoint` que guardará el modelo en el archivo definido por `filepath`. Este callback monitorea la métrica de `loss` (pérdida), y guardará el modelo solo cuando se observe una mejora en la pérdida mínima (`mode='min'`). El parámetro `verbose=1` asegura que se imprima información sobre cada checkpoint guardado.

- `callbacks_list = [checkpoint]`
  - Finalmente, se crea una lista de callbacks que incluye el checkpoint definido. Esta lista puede ser pasada al método de entrenamiento del modelo (`model.fit`) para habilitar el guardado automático del modelo durante el entrenamiento.

Este proceso permite guardar los pesos del modelo en cada época donde se observa una mejora en la pérdida, asegurando que siempre se conserve la mejor versión del modelo durante el entrenamiento.

In [55]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

**Entrenar el Modelo LSTM:**

- `model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)`
  - Esta línea de código entrena el modelo LSTM utilizando los datos de entrada `X` y las etiquetas `y`.
  - `epochs=20` especifica que el modelo se entrenará durante 20 épocas.
  - `batch_size=128` indica que el tamaño del lote para el entrenamiento será de 128 muestras.
  - `callbacks=callbacks_list` incluye la lista de callbacks que hemos definido, en este caso, el checkpoint. Esto asegura que el modelo guarde los pesos que mejoran la pérdida durante el entrenamiento.

Este proceso inicia el entrenamiento del modelo LSTM, ajustando sus pesos con los datos de entrenamiento a lo largo de las épocas especificadas y utilizando los callbacks para guardar el mejor modelo basado en la métrica de pérdida.

In [56]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 3.0820
Epoch 1: loss improved from inf to 2.96101, saving model to weights-improvement-01-2.9610.keras
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - loss: 3.0819
Epoch 2/20
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.7178
Epoch 2: loss improved from 2.96101 to 2.67133, saving model to weights-improvement-02-2.6713.keras
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 42s 31ms/step - loss: 2.7177
Epoch 3/20
1279/1280 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.5424
Epoch 3: loss improved from 2.67133 to 2.51534, saving model to weights-improvement-03-2.5153.keras
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 39s 30ms/step - loss: 2.5424
Epoch 4/20
1279/1280 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.4103
Epoch 4: loss improved from 2.51534 to 2.38243, saving model to weights-improvement-04-2.3824.keras
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 39s 30ms/step - loss: 2.4102
Epoch 5/20
1279/1280 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.2917
Epoch 5: loss 

**Cargar los Pesos de la Red:**

- `filename = "weights-improvement-.keras"`
  - Define el nombre del archivo que contiene los pesos del modelo previamente guardados. En este caso, el archivo se llama "weights-improvement-19-2.6125.keras", lo que sugiere que corresponde a la época 19 con una pérdida de 2.6125.

- `model.load_weights(filename)`
  - Esta línea carga los pesos del modelo desde el archivo especificado por `filename`. Esto restablece el estado del modelo a los pesos que fueron guardados anteriormente.

- `model.compile(loss='categorical_crossentropy', optimizer='adam')`
  - Compila el modelo nuevamente después de cargar los pesos. Utiliza la función de pérdida `categorical_crossentropy` y el optimizador `adam`. Este paso asegura que el modelo esté listo para hacer predicciones o continuar con el entrenamiento, si es necesario.

Este código restaura el modelo a un estado previamente guardado, permitiendo utilizar los pesos optimizados para hacer predicciones o seguir con el entrenamiento a partir de ese punto.

In [58]:
# load the network weights
filename = "weights-improvement-20-1.6581.keras"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

**Crear Mapeo de Enteros a Caracteres:**

- `int_to_char = dict((i, c) for i, c in enumerate(chars))`
  - Esta línea de código crea un diccionario `int_to_char` que mapea cada entero (`i`) a su correspondiente carácter (`c`). Utiliza la función `enumerate(chars)` para obtener pares de índice y carácter de la lista `chars`, y luego construye el diccionario mediante una comprensión de diccionario. Este mapeo es útil para convertir las predicciones numéricas del modelo de vuelta a caracteres legibles, facilitando la generación de texto.

In [59]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

**Elegir una Semilla Aleatoria y Generar Caracteres:**

- `start = np.random.randint(0, len(dataX)-1)`
  - Selecciona un índice aleatorio `start` dentro del rango de índices válidos en `dataX`.

- `pattern = dataX[start]`
  - Utiliza el índice aleatorio `start` para seleccionar una secuencia de entrada (`pattern`) de `dataX`.

- `print("Seed:")`
  - Imprime el mensaje "Seed:".

- `print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")`
  - Convierte la secuencia de enteros `pattern` en una cadena de caracteres legible utilizando el diccionario `int_to_char` y la imprime como la semilla inicial.

**Generar Caracteres:**

- Bucle `for i in range(1000)`:
  - Inicia un bucle para generar 1000 caracteres adicionales.

  - `x = np.reshape(pattern, (1, len(pattern), 1))`
    - Reformatea `pattern` para que tenga las dimensiones adecuadas para el modelo: `[1, len(pattern), 1]`.

  - `x = x / float(n_vocab)`
    - Normaliza `x` dividiendo por el tamaño del vocabulario (`n_vocab`), escalando los valores entre 0 y 1.

  - `prediction = model.predict(x, verbose=0)`
    - Utiliza el modelo entrenado para predecir el siguiente carácter basado en la secuencia `x`. `verbose=0` suprime la salida detallada de la predicción.

  - `index = np.argmax(prediction)`
    - Encuentra el índice del valor más alto en la predicción, que corresponde al siguiente carácter predicho.

  - `result = int_to_char[index]`
    - Convierte el índice `index` en el carácter correspondiente utilizando el diccionario `int_to_char`.

  - `seq_in = [int_to_char[value] for value in pattern]`
    - Convierte la secuencia actual de enteros `pattern` en caracteres legibles.

  - `print(result, end = "")`
    - Imprime el carácter predicho sin salto de línea (`end = ""`), continuando la salida en la misma línea.

  - `pattern.append(index)`
    - Añade el índice predicho `index` al final de la secuencia `pattern`.

  - `pattern = pattern[1:len(pattern)]`
    - Ajusta `pattern` para eliminar el primer carácter y mantener la longitud de la secuencia.

- `print("\nDone.")`
  - Imprime "Done." al final del proceso de generación de texto.

Este código selecciona una semilla aleatoria de caracteres, luego utiliza el modelo LSTM para generar 1000 caracteres adicionales, continuando la secuencia basada en las predicciones del modelo.

In [60]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(result, end = "")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" by violently up and down, and the poor little thing howled so,
that alice could hardly hear the word "
s her head of the goustent of the garter, and the mouse was a little goudot to the sabbit was the dourt, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the gale of the gound of the gouse, and then she was not all the mouse of the ga